In [ ]:
#integration functions
import random
from skhep.math.vectors import LorentzVector, Vector3D
import math
self=model
def integrate(br, coupling, m0, m1, m2, m3, nsample):

    # prepare output
    particles, weights = [], []
    #create parent 4-vector
    p_mother=LorentzVector(0,0,0,m0)

    #integration boundary
    q2min,q2max = (m2+m3)**2,(m0-m1)**2
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):

        #Get kinematic Variables
        q2 = random.uniform(q2min,q2max)
        q  = math.sqrt(q2)
        E2st = (q**2 - m2**2 + m3**2)/(2*q)
        E3st = (m0**2 - q**2 - m1**2)/(2*q)
        m232min = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) + np.sqrt(E3st**2 - m1**2))**2
        m232max = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) - np.sqrt(E3st**2 - m1**2))**2
        cthmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        cthmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        ENmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        ENmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)
        #th = np.arccos(EN)


        #decay meson and V
        #cosQ =random.uniform(-1,1)
        #phiQ =random.uniform(-math.pi,math.pi)
        #cosM =random.uniform(-1.,1.)
        #phiM =random.uniform(-math.pi,math.pi)
        #branching fraction
        brval  = eval(br)
        #print("q val ",q," EN val ",EN," Br ",brval, " Br ",eval(br))
        brval *= (q2max-q2min)*(cthmax-cthmin)/float(nsample)

        #save
        #particles.append(p_3)
        weights.append(brval)
    #print('branching frac', brval)
    #return particles,weights
    return(sum(weights))

def integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample):
    m0=self.masses(f'{pid0}')
    m1=self.masses(f'{pid1}')
    m2=self.masses(f'{pid2}')
    integ=integrate(dbr, 1, m0, m1, m2,m3, nsample)
    return(integ)
    
def integrate_EN(br, coupling, m0, m1, m2, m3, nsample):
    #prepare output
    weights=[]
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):
        #get energy bounds
        ENmin=m3
        ENmax=(m0**2+m3**2-m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)

        #branching fraction
        brval  = eval(br)
        brval *= (ENmax-ENmin)/float(nsample)

        #save
        weights.append(brval)
    return(sum(weights))


#pid0 is parent particle, pid1 is produced meson pid2 is the smaller particle
#
from matplotlib import pyplot as plt
self=model
def show_br_curve(br,pid0,pid1,pid2,integration,nsample=100):
    if integration=='br':
        coupling=1
        delm=.1
        mass0=0
        mass=0.01
        x=[]
        y=[]
        for n in range(1,int((3-mass0)/delm)):
            y.append(eval(br))
            x.append(mass)
            mass+=delm

        plt.plot(x,y)
        plt.xlim([0,3])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")

    if integration=='dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        m3=0
        qmin,qmax=(m2+m3),(m0-m1)
        coupling=1
        delm=.1
        x=[]
        y=[]
        for n in range(1,30):
            y.append(integrate_EN(dbr, 1, m0, m1, m2,m3, nsample))
            x.append(m3)
            m3+=delm
        plt.xlim([0,2])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    if integration=='dq2dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        coupling=1
        delm=.1
        x=[]
        y=[]
        m3=0
        for n in range(1,30):
            y.append(integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample))
            #y.append(integrate_pseud_3body(df,pid0,pid1,pid2,m3,1,nsample=nsample,channel="D"))
            x.append(m3)
            m3+=delm
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    plt.show()


import pandas as pd

#n can be: 2 for 2-body and 3 for 3-body. type can be: 2 body regular, 2 body tau to pi, 2 body tau to K, 2 body tau to rho, 3 body pseudo, 3 body vector, 3 body tau
#pid0 is parent pid1 is produces meson pid2 is the smaller particle, usually the electron
#for 2 body pid0 is the parent and pid1 is the produced SM particle
def add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=10,generator="Pythia8",n=2,type_decay="2 body regular",show_br=True):
    if n==2 and type_decay=="2 body regular":
        df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
        particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        br=hnl.get_2body_br(pid0,pid1)
        model.add_production_2bodydecay(
        pid0 = f"{pid0}",
        pid1 = f"{pid1}",
        br=br,
        generator = generator,
        energy = energy,
        nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        #show_br_curve(df,br,pid0,pid1=None,pid2=pid2,decay_type='2 body',nsample=nsample)
        
    if n==2 and type_decay=="2 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','process'])
        particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]

        if pid1=="-213" or pid1=="213":
            br=hnl.get_2body_br_tau_rho(pid0,pid1)
        else:
            br=hnl.get_2body_br_tau_H(pid0,pid1)

        model.add_production_2bodydecay(
            pid0 = f"{pid0}",
            pid1 = f"{pid1}",
            br=br,
            generator = generator,
            energy = energy,
            nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        
    #here pid1 is the electron (lepton) now and pid2 is the neutrino
    if n==3 and type_decay=="3 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
        particles=[['15','11','18','tau^- -> e^- + nu_tau']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_tau(pid0,pid1,pid2)
        model.add_production_3bodydecay(
            label= "5_di",
            pid0 = pid0,
            pid1 = pid1,
            pid2 = pid2,
            br = dbr,
            generator ="pythia8",
            energy = energy,
            nsample = nsample,
            scaling = 0, 
        )
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dEN",nsample=nsample)
        
    if n==3 and type_decay=="3 body pseudo":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
        ['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
        ['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
        ['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
        ['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
        ['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
        ['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
        ['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_pseudoscalar(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)

    if n==3 and type_decay=="3 body vector":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_vector(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)


#for testing functions
#for 2 body decays
df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

#for 2 body decays tau
df_t=pd.DataFrame([],columns=['pid0','pid1','process'])
particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
for n in range(len(particles)):
    df_t.loc[len(df_t)]=particles[n]

#for 3 body decays tau
df_3t=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
particles=[['15','11','18','tau^- -> e^- + nu_tau + N'],['15','11','-12','tau^- -> e^- + \bar{nu}_e + N']]
for n in range(len(particles)):
    df_3t.loc[len(df_3t)]=particles[n]

#for 3 body pseudo
df_D=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
for n in range(len(particles)):
    df_D.loc[len(df_D)]=particles[n]

#for 3 body vector
df=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],
['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],
['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],
['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],
['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],
['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

pid0='421'
pid1='-323'
pid2='-11'

add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=100,generator="Pythia8",n=3,type_decay="3 body vector",show_br=True)